In [1]:
import datetime
import requests
import hyp3_sdk
import numpy as np
from tqdm import tqdm 
from copy import deepcopy
import requests
import itertools
from multiprocessing import Pool

In [2]:
%load_ext autoreload
%autoreload 2

In [96]:
def isce_jobs4resubmit(jobs_dicts:list):
    job_definition = {'name': None,
                      'job_parameters': None,
                      'job_type': 'INSAR_ISCE'
                     }

    prepared_jobs = []
    for job in jobs_dicts:
        prepared_job = deepcopy(job_definition)
        prepared_job['name'] = job['name']
        prepared_job['job_parameters'] = job['job_parameters']
        prepared_job['job_type'] = job['job_type']
        prepared_jobs.append(prepared_job)
    return prepared_jobs

def raider_jobs4resubmit(jobs_dicts:list):
    job_definition = {
        'name': 're-run RAiDER step',  # optional: provide a name
        "job_type": "ARIA_RAIDER",
        "job_parameters": {
            # "job_id": "27836b79-e5b2-4d8f-932f-659724ea02c3",
            # "weather_model": "ERA5"
        },
    }

    prepared_jobs = []
    for job in jobs_dicts:
        prepared_job = deepcopy(job_definition)  
        if job['job_type'] == 'INSAR_ISCE':
            prepared_job['name'] = job['name'] + '_rerun_raider'
            prepared_job['job_parameters']['job_id'] = job['job_id']
        else:
            prepared_job['name'] = job['name'] 
            prepared_job['job_parameters']['job_id'] = job['job_parameters']['job_id']
        prepared_job['job_parameters']['weather_model'] = job['job_parameters']['weather_model'] 
        prepared_jobs.append(prepared_job)
    return prepared_jobs

def split_failed_jobs(failed_jobs):
    flag_insar_job = lambda x, y: (x.job_type == 'INSAR_ISCE') & (len(x.processing_times)==y)
    # isce
    isce_jobs = [j.to_dict() for j in failed_jobs if flag_insar_job(j, 1)]
    # raider
    raider_jobs_new = [j.to_dict() for j in failed_jobs if flag_insar_job(j, 2)] 
    raider_jobs = [j.to_dict() for j in failed_jobs if j.job_type == 'ARIA_RAIDER']
    raider_jobs = raider_jobs_new + raider_jobs
    return isce_jobs, raider_jobs

def get_request_date(job_dict):
    job_date = datetime.datetime.strptime(job_dict['request_time'], '%Y-%m-%dT%H:%M:%S%z')
    return datetime.datetime.strftime(job_date,'%Y-%m-%d') 

# TODO: add function to find duplicate jobs to filter out same jobs
# for now track it by date
class access_hyp3_resubmit():
    def __init__(self, hyp3_jobs, verbose=True):
        # Filter to failed jobs
        failed_jobs = hyp3_jobs.filter_jobs(succeeded=False, pending=False,
                                            running=False, failed=True)
        print(f'Failed jobs:\n {failed_jobs}')

        # Failed at topssApp step
        self.isce_jobs, self.raider_jobs = split_failed_jobs(failed_jobs)
        if verbose:
            self.print_info()

    def print_info(self):
        print(f'Topsapp jobs: {len(self.isce_jobs)}')
        print(f' Submission dates: {np.unique(list(map(get_request_date, self.isce_jobs)))}')

        # Failed at raider step
        print(f'Raider jobs: {len(self.raider_jobs)}')
        print(f' Submission dates: {np.unique(list(map(get_request_date, self.raider_jobs)))}')

    def prepare_jobs(self, batch_size=200):
        self.prepared_isce = isce_jobs4resubmit(self.isce_jobs)
        self.prepared_raider = raider_jobs4resubmit(self.raider_jobs)
        # isce
        if len(self.prepared_isce) > 0:
            self.isce_resubmit = [batch for batch in hyp3_sdk.util.chunk(self.prepared_isce, batch_size)]
            msg = f'Prepared isce jobs: {len(self.prepared_isce)} in {len(self.isce_resubmit[0])}'
            msg += f'x {len(self.isce_resubmit)} batches'
            print(msg)
        else:
            self.isce_resubmit = None 
        # raider
        if len(self.prepared_raider) > 0:
            self.raider_resubmit = [batch for batch in hyp3_sdk.util.chunk(self.prepared_raider, batch_size)]
            msg = f'Prepared raider jobs: {len(self.prepared_raider)} in {len(self.raider_resubmit[0])}'
            msg += f'x {len(self.raider_resubmit)} batches'
            print(msg)
        else:
            self.raider_resubmit = None
        
    def submit_isce(self):
        if self.isce_resubmit is None:
            raise ValueError('There are no prepared jobs!') 
        print(f'Submitting: {len(self.prepared_isce)} jobs')
        for batch in tqdm(self.isce_resubmit):
            hyp3_isce.submit_prepared_jobs(batch)

    def submit_raider(self):
        if self.raider_resubmit is None:
            raise ValueError('There are no prepared jobs!') 
        print(f'Submitting: {len(self.prepared_raider)} jobs')
        for batch in tqdm(self.raider_resubmit):
            hyp3_isce.submit_prepared_jobs(batch) 


In [252]:
## Filtering functions
def _exist(failed_job, filt_job_list):
    return failed_job['job_parameters'] in filt_job_list

# Filter based on status code
def filter_failed_jobs(failed_jobs:list, jobs: hyp3_sdk.jobs.Batch, n_jobs:int=10, verbose:bool = False):
    # Get succeded, running and pending
    filt_job_list = [j.job_parameters for j in jobs.filter_jobs().jobs]
    if verbose: print(f'Succedded,pending,running jobs: {len(filt_job_list)}')
    if verbose: print(f'Failed jobs: {len(failed_jobs)}')
    
    # Check if exists in parallel 
    with Pool(n_jobs) as pool:
        flags = pool.starmap(_exist, zip(failed_jobs, itertools.repeat(filt_job_list)))
    if verbose: print(f'Failed jobs that exist in above: {np.count_nonzero(np.atleast_1d(flags))}')

    # Filter 
    filt_failed = list(itertools.compress(failed_jobs, ~np.bool_(flags)))
    if verbose: print(f'Number of filtered failed jobs: {len(filt_failed)}')

    return filt_failed

# Filter duplicates
# NOTE: wrote this fast, probably could be done better
request_date2dt = lambda x: datetime.datetime.strptime(x['request_time'],
                                                         '%Y-%m-%dT%H:%M:%S%z')
def find_latest_datetime_index(datetimes):
    if not datetimes:
        return None
    
    latest_index = 0
    latest_datetime = datetimes[0]
    
    for i, dt in enumerate(datetimes[1:], start=1):
        if dt > latest_datetime:
            latest_datetime = dt
            latest_index = i
    
    return latest_index

def get_job_last_requested(duplicate_jobs: list): 
    dates = [request_date2dt(d) for d in duplicate_jobs]
    ix = find_latest_datetime_index(dates)
    return duplicate_jobs[ix]

 
def find_duplicates(job_list: list):
    # strip to only job_parameters
    jobs2count = [j['job_parameters'] for j in job_list]

    # Get count for jobs
    count = {} 
    for ix, j1 in enumerate(jobs2count):
        index = []
        for ik, j2 in enumerate(jobs2count[ix:]):
            if j1 == j2:
                index.append(ik)
        count[ix] = index

    # Filter out same duplicates
    for ix in list(count):
        if count.get(ix, None) is not None: 
            for ik in count.get(ix, None):
                if ik != ix:
                    count.pop(ik, None)

    # Loop through duplicates and get job last request
    filt_jobs = []
    for ix in count:
        duplicates =  [job_list[i] for i in count[ix]]
        if len(duplicates) > 1:
            filt_jobs.append(get_job_last_requested(duplicates))
        else:
            filt_jobs.append(duplicates[0])
    return filt_jobs

# Filter based on exit log message
# example when job failed at raider step due to requested model that does not exist in that aoi
hrr_warning_msg = 'ValueError: HRRR was requested but it is not available in this area'

def get_log_exit_msg(log_url, msg):
    response = requests.get(log_url, headers = {'Range': 'bytes=-1000'})
    return response.text.split('\n')[-1] == msg

# Example notebook to inspect jobs: https://github.com/ACCESS-Cloud-Based-InSAR/Aria-Hyp3-GUNW-Frame-Submission/blob/dev/operational_scripts/__Andrew_Inspection.ipynb

## Setup
example on jobs submitted for NISAR calval

In [3]:
# uses .netrc; add `prompt=True` to prompt for credentials;
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu', prompt=True)
# hyp3_isce = hyp3_sdk.HyP3('https://hyp3-tibet-jpl.asf.alaska.edu', prompt=True)

In [4]:
filtering_flags = dict(succeeded=False, pending=False,
                        running=False, failed=True)

first jobs were submitted on May 8

In [187]:
# May-08 to May 09 

# request separate to differeniate between topsApp and raider jobs
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 8), 
                           end=datetime.datetime(2024, 5, 9),
                           user_id='access_cloud_based_insar',
                           job_type='INSAR_ISCE')
print(jobs)
print('#' * 10)
jobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

raider_jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 5, 8), 
                           end=datetime.datetime(2024, 5, 9),
                           job_type='ARIA_RAIDER')
print('#' * 10)
print('Raider')
print(raider_jobs)

63861 HyP3 Jobs: 14695 succeeded, 49166 failed, 0 running, 0 pending.
##########
Failed jobs:
 49166 HyP3 Jobs: 0 succeeded, 49166 failed, 0 running, 0 pending.
Topsapp jobs: 45687
 Submission dates: ['2024-05-08']
Raider jobs: 3392
 Submission dates: ['2024-05-08']
##########
Raider
0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.


in total, 63861 jobs were submitted, 49166 failed of which 45687 failed at topsApp step and 3392 at raider step

In [214]:
# second run
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 10), 
                           end=datetime.datetime(2024, 5, 15),
                           user_id='access_cloud_based_insar',
                           job_type='INSAR_ISCE')
print(jobs)
print('#' * 10)
jobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

raider_jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 5, 10), 
                           end=datetime.datetime(2024, 5, 15),
                           job_type='ARIA_RAIDER')
print('#' * 10)
print('Raider')
print(raider_jobs)

45687 HyP3 Jobs: 36170 succeeded, 9517 failed, 0 running, 0 pending.
##########
Failed jobs:
 9517 HyP3 Jobs: 0 succeeded, 9517 failed, 0 running, 0 pending.
Topsapp jobs: 8390
 Submission dates: ['2024-05-11']
Raider jobs: 1011
 Submission dates: ['2024-05-11']
##########
Raider
3392 HyP3 Jobs: 0 succeeded, 3392 failed, 0 running, 0 pending.


after second run we have 50865 of 63861

In [219]:
# unique succeded aria_raider jobs
np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**{**filtering_flags, **{'succeeded':True, 'failed':False}}).jobs]).shape

(0,)

In [220]:
# unique failed aria_raider jobs
np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**filtering_flags).jobs]).shape

(3392,)

## Prepare for re-submission

In [217]:
rjobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 10), 
                           end=datetime.datetime(2024, 5, 15),
                           user_id='access_cloud_based_insar')
print(rjobs)
print('#' * 10)
rjobs = access_hyp3_resubmit(rjobs.filter_jobs(**filtering_flags))

49079 HyP3 Jobs: 36170 succeeded, 12909 failed, 0 running, 0 pending.
##########
Failed jobs:
 12909 HyP3 Jobs: 0 succeeded, 12909 failed, 0 running, 0 pending.
Topsapp jobs: 8390
 Submission dates: ['2024-05-11']
Raider jobs: 4403
 Submission dates: ['2024-05-11']


### Get raider jobs  with wrong weather model submission

In [218]:
# I submitted HRRR over Hawaii and Alaska where is not HRRR
logs = [j['logs'][0] for j in rjobs.raider_jobs if len(j['logs']) > 0]

with Pool(10) as pool:
    # Map the function over the logs, passing the warning message to each call
    job_flags = pool.starmap(get_log_exit_msg, zip(logs, itertools.repeat(hrr_warning_msg)))

print(f'Number of failed raider jobs: {len(list(itertools.compress(logs, job_flags)))} of {len(logs)}')
print(f' with msg: {hrr_warning_msg}')

Number of failed raider jobs: 4201 of 4320
 with msg: ValueError: HRRR was requested but it is not available in this area


In [224]:
# Find jobs that failed with defined msg
nohrrr_jobs = list(itertools.compress(rjobs.raider_jobs, job_flags))

# Find tracks, tracks 124, 87 (Hawaii), 44 (Northern Alaska Slopes)
np.unique([j['name'] for j in nohrrr_jobs])

array(['nisar_calval_hrr-track124',
       'nisar_calval_hrr-track124_rerun_raider',
       'nisar_calval_hrr-track44_rerun_raider',
       'nisar_calval_hrr-track87',
       'nisar_calval_hrr-track87_rerun_raider'], dtype='<U38')

In [236]:
import pandas as pd
# Get job_ids to give ASF for manual import to DAAC
hrrr_jobs = []

for j in nohrrr_jobs:
    if j['job_type'] == 'INSAR_ISCE':
        hrrr_jobs.append(j['job_id'])
    elif j['job_type'] == 'ARIA_RAIDER':
        hrrr_jobs.append(j['job_parameters']['job_id'])

hrrr_jobs = pd.DataFrame(hrrr_jobs, columns=['job_id'])
hrrr_jobs.head()

,job_id
0,08291364-1f89-49d1-b8ee-595105607eaa
1,418b5ed3-19f7-4809-925a-27ea193b9196
2,1384751d-8705-40d4-abdd-8315564202e6
3,f3585c34-b340-43ad-9fdd-d5c0193996fb
4,0c47b1db-ada3-4975-85b7-87051a4f0a6f


In [238]:
hrrr_jobs.to_csv('access_nohrrr_job_ids.csv', index=False)

In [245]:
# To avoid submitting this jobs again, filter them out
job_flags2 = [~np.bool_(jf) for jf in job_flags]
rjobs.raider_jobs = list(itertools.compress(rjobs.raider_jobs, job_flags2))

In [247]:
# To submit
rjobs.print_info()

Topsapp jobs: 8390
 Submission dates: ['2024-05-11']
Raider jobs: 119
 Submission dates: ['2024-05-11']


### Filtering to avoid tracking submission dates
NOTE: need to work on this, does not filter correctly

In [267]:
# Last run
all_jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 8), 
                           end=datetime.datetime(2024, 5, 15),
                           user_id='access_cloud_based_insar')
print(all_jobs)
print('#' * 10)
access_jobs = access_hyp3_resubmit(all_jobs.filter_jobs(**filtering_flags))

####### Filter failed jobs that succeeded, are pending or running ####### 
print('#' * 10)
verbose = False
if verbose: print('Failed jobs at topsApp step')
access_jobs.isce_jobs = filter_failed_jobs(access_jobs.isce_jobs, all_jobs,
                                           n_jobs=20, verbose=verbose)
if verbose: print('\nFailed jobs at raider step')
access_jobs.raider_jobs = filter_failed_jobs(access_jobs.raider_jobs, all_jobs,
                                             n_jobs=20, verbose=verbose)

print('\nAfter filtering already succedded, pending, running')
print('#' * 10)
access_jobs.print_info()

112940 HyP3 Jobs: 50865 succeeded, 62075 failed, 0 running, 0 pending.
##########
Failed jobs:
 62075 HyP3 Jobs: 0 succeeded, 62075 failed, 0 running, 0 pending.
Topsapp jobs: 54077
 Submission dates: ['2024-05-08' '2024-05-11']
Raider jobs: 7795
 Submission dates: ['2024-05-08' '2024-05-11']
##########


In [ ]:
def find_duplicates(job_list: list):
    # strip to only job_parameters
    jobs2count = [j['job_parameters'] for j in job_list]

    # Get count for jobs
    count = {} 
    for ix, j1 in enumerate(jobs2count):
        index = []
        for ik, j2 in enumerate(jobs2count):
            if (j1 == j2):
                index.append(ik)
        count[ix] = index

    # Filter out same duplicates
    for ix in list(count):
        if count.get(ix, None) is not None: 
            for ik in count.get(ix, None):
                if ik != ix:
                    count.pop(ik, None)

    # Loop through duplicates and get job last request
    filt_jobs = []
    for ix in count:
        duplicates =  [job_list[i] for i in count[ix]]
        if len(duplicates) > 1:
            filt_jobs.append(get_job_last_requested(duplicates))
        else:
            filt_jobs.append(duplicates[0])
    return filt_jobs, count

In [262]:
####### Filter duplicated failed jobs ####### 
## NOTE: think this part does not work right
access_jobs.isce_jobs = find_duplicates(access_jobs.isce_jobs)
access_jobs.raider_jobs = find_duplicates(access_jobs.raider_jobs)

print('\nAfter filtering failed jobs duplicates')
print('#' * 10)
access_jobs.print_info()


After filtering failed jobs duplicates
##########
Topsapp jobs: 12278
 Submission dates: ['2024-05-11']
Raider jobs: 1
 Submission dates: ['2024-05-11']


In [265]:
a, b = find_duplicates(access_jobs.isce_jobs)

In [266]:
b

{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
 

## Re-submission

In [248]:
# Prepare jobs for re-submission
rjobs.prepare_jobs()

Prepared isce jobs: 8390 in 200x 42 batches
Prepared raider jobs: 119 in 119x 1 batches


In [249]:
# this goes faster
rjobs.submit_raider()

Submitting: 119 jobs


100%|██████████| 1/1 [00:03<00:00,  3.55s/it]


In [250]:
rjobs.submit_isce()

Submitting: 8390 jobs


100%|██████████| 42/42 [01:14<00:00,  1.77s/it]


In [337]:
succeded_flag = {**filtering_flags, **{'succeeded':True, 'failed':False}}
unique_succeded = np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**succeded_flag).jobs])
unique_failed = np.unique([j.job_parameters['job_id'] for j in raider_jobs.filter_jobs(**filtering_flags).jobs])
# NOTE compare ARIA_RAIDER with INSAR_ISCE job_id

In [256]:
# Check
jobs = hyp3_isce.find_jobs(start=datetime.datetime(2024, 5, 15), 
                           end=datetime.datetime(2024, 5, 18),
                           user_id='access_cloud_based_insar',
                           job_type='INSAR_ISCE')
print(jobs)
print('#' * 10)
jobs = access_hyp3_resubmit(jobs.filter_jobs(**filtering_flags))

raider_jobs = hyp3_isce.find_jobs(user_id='access_cloud_based_insar',
                           start=datetime.datetime(2024, 5, 15), 
                           end=datetime.datetime(2024, 5, 18),
                           job_type='ARIA_RAIDER')
print('#' * 10)
print('Raider')
print(raider_jobs)

8390 HyP3 Jobs: 0 succeeded, 0 failed, 915 running, 7475 pending.
##########
Failed jobs:
 0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.
Topsapp jobs: 0
 Submission dates: []
Raider jobs: 0
 Submission dates: []
##########
Raider
119 HyP3 Jobs: 0 succeeded, 31 failed, 88 running, 0 pending.


In [255]:
raider_jobs.filter_jobs(**filtering_flags).jobs

[Job.from_dict({'job_id': '115a376c-cc05-46a5-af5a-a564fbe93565', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-17T00:22:39+00:00', 'status_code': 'FAILED', 'user_id': 'access_cloud_based_insar', 'name': 'nisar_calval_hrr-track87_rerun_raider', 'job_parameters': {'job_id': '23f4d608-6df9-4ed9-afa4-ac9a40c75346', 'weather_model': 'HRRR'}, 'logs': ['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/115a376c-cc05-46a5-af5a-a564fbe93565/115a376c-cc05-46a5-af5a-a564fbe93565.log'], 'expiration_time': '2024-11-14T00:00:00+00:00', 'processing_times': [7.295], 'credit_cost': 1}),
 Job.from_dict({'job_id': '4a4de9e7-fbb3-4db5-b3f6-8bb9022bab34', 'job_type': 'ARIA_RAIDER', 'request_time': '2024-05-17T00:22:39+00:00', 'status_code': 'FAILED', 'user_id': 'access_cloud_based_insar', 'name': 'nisar_calval_hrr-track87_rerun_raider', 'job_parameters': {'job_id': '6356092d-daca-4fe5-957d-ec14c115d1e3', 'weather_model': 'HRRR'}, 'logs': ['https://hyp3-a19-jpl-contentbucket

In [330]:
## Check failed jobs
print(f'# failed jobs {len(rjobs.raider_jobs)}')
all_logs = [j['logs'] for j in rjobs.raider_jobs]
all_logs[0], all_logs[-1]

# failed jobs 27160


(['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/91d2ad06-b84c-4d88-bfdf-b5881c3684ed/91d2ad06-b84c-4d88-bfdf-b5881c3684ed.log'],
 ['https://hyp3-a19-jpl-contentbucket-1wfnatpznlg8b.s3.us-west-2.amazonaws.com/9d27256a-acf3-4ed9-92b0-c0d04374a6c8/9d27256a-acf3-4ed9-92b0-c0d04374a6c8.log'])

## asf search

In [8]:
import asf_search as asf
import geopandas as gpd

In [201]:
# requires asf_search > 7.0.8
opts = asf.ASFSearchOptions(
    #shortName='ARIA_S1_GUNW', # version 3
    #relativeOrbit=[13],
    #flightDirection='DESCENDING',
    processingLevel='GUNW_STD',
    intersectsWith='POLYGON((-127.992 32.2865,-115.5115 32.2865,-115.5115 49.1812,-127.992 49.1812,-127.992 32.2865))',
    collectionAlias=False,
)

In [9]:
# requires asf_search > 7.0.8
opts = asf.ASFSearchOptions(
    shortName='ARIA_S1_GUNW', # version 3
    #relativeOrbit=[13],
    #flightDirection='DESCENDING',
    #processingLevel='GUNW_STD',
    intersectsWith='POLYGON((-118.5619 33.4925,-117.6485 33.4925,-117.6485 34.2148,-118.5619 34.2148,-118.5619 33.4925))',
    collectionAlias=False,
)

In [10]:
scenes = asf.search(opts=opts)
gdf = gpd.GeoDataFrame.from_features(scenes.geojson(), crs='EPSG:4326')

In [11]:
gdf.shape

(5418, 30)

In [25]:
gdf[gdf.fileID.apply(lambda x: x.split('-')[6]) == '20190123_20190117'].url.values

array(['https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-064-tops-20190123_20190117-014931-00119W_00033N-PP-cc61-v3_0_1.nc'],
      dtype=object)

In [12]:
version = gdf.fileID.apply(lambda x: int(x.split('-')[-1].split('_')[0].split('v')[-1]))
version.unique()

array([3])